<a href="https://colab.research.google.com/github/mrzebest/MachineDeepLearning/blob/main/ReconnaissanceObjets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Librairies
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harishkumardatalab/food-image-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/food-image-classification-dataset


In [ ]:
# Déclaration du chemin
DATASET_PATH = './dataset'

In [ ]:

!mkdir -p {DATASET_PATH} # Création du chemin

!cp -r {path}/* {DATASET_PATH}/ # Attribution

In [ ]:
# Chemin des dossiers racines
base_dir = '/content/dataset/Food Classification dataset/'

# Paramètres
img_size = (64, 64)
batch_size = 32

# Création des générateurs d'images
datagen = ImageDataGenerator(rescale=1./255)

# Chargement des données depuis les répertoires
def create_dataframe(directory, label):
    generator = datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=False
    )

    filepaths = generator.filepaths
    labels = generator.labels
    df = pd.DataFrame({'image': filepaths, 'label': labels})
    return df, generator

In [ ]:
df, generator = create_dataframe(base_dir,'label')

print(df)

Found 23873 images belonging to 34 classes.
                                                   image  label
0      /content/dataset/Food Classification dataset/B...      0
1      /content/dataset/Food Classification dataset/B...      0
2      /content/dataset/Food Classification dataset/B...      0
3      /content/dataset/Food Classification dataset/B...      0
4      /content/dataset/Food Classification dataset/B...      0
...                                                  ...    ...
23868  /content/dataset/Food Classification dataset/s...     33
23869  /content/dataset/Food Classification dataset/s...     33
23870  /content/dataset/Food Classification dataset/s...     33
23871  /content/dataset/Food Classification dataset/s...     33
23872  /content/dataset/Food Classification dataset/s...     33

[23873 rows x 2 columns]


In [ ]:
#Librairie nécessaire au chargement d'images
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [ ]:
# Convertir le dataframe en entrées, X rappelez vous, donc les vraies images loadées et non les paths, et y (labels)
# Pour ce faire, on utilisera : load_img, et , img_to_array
def load_images(df, img_size):
    X = []
    for path in df['image']:
        img = load_img(path, target_size=img_size)
        img_array = img_to_array(img) / 255.0
        X.append(img_array)
    return np.array(X)

X = load_images(df, img_size)
y = to_categorical(df['label'])

# Séparation des données d'entrainement et des données de test, en utilisant une méthode de scickit learn
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=df['label'], random_state=42)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [ ]:
# Modélisation :
from tensorflow.keras import models, layers

model = models.Sequential([
    # 1er bloc convolutionnel
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),  # 32 filtres 3x3 + fonction ReLU
    layers.MaxPooling2D((2, 2)),  # Réduction de la taille par 2

    # 2e bloc convolutionnel
    layers.Conv2D(64, (3, 3), activation='relu'),  # 64 filtres 3x3
    layers.MaxPooling2D((2, 2)),

    # 3e bloc convolutionnel
    layers.Conv2D(64, (3, 3), activation='relu'),

    # Flatten + couches denses
    layers.Flatten(),  # Aplatissement des cartes de caractéristiques
    layers.Dense(64, activation='relu'),  # Couche dense avec 64 neurones
    layers.Dense(34, activation='softmax')  # Couche de sortie : 34 classes → 34 scores (logits)
])

In [ ]:
# Compilation du modèle
model.compile(optimizer='adam',  # Algorithme d’optimisation
              loss='categorical_crossentropy',  # Fonction de perte pour classification multi-classe (pour les labels one-hot)
              metrics=['accuracy'])  # Évaluation avec la précision

# Affichage du résumé du modèle
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       589,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 34)             │         2,210 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 648,418 (2.47 MB)

 Trainable params: 648,418 (2.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Entraînement du modèle (fit)
history = model.fit(X_train, y_train, epochs=10,  # Nombre d'époques
                    validation_data=(X_val, y_val))  # Validation sur données de test


Epoch 1/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 111s 186ms/step - accuracy: 0.5965 - loss: 1.3400 - val_accuracy: 0.3468 - val_loss: 2.5689
Epoch 2/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 141s 185ms/step - accuracy: 0.6546 - loss: 1.1502 - val_accuracy: 0.3520 - val_loss: 2.8226
Epoch 3/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 114s 191ms/step - accuracy: 0.7192 - loss: 0.9397 - val_accuracy: 0.3399 - val_loss: 3.0256
Epoch 4/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 138s 185ms/step - accuracy: 0.7703 - loss: 0.7706 - val_accuracy: 0.3447 - val_loss: 3.2998
Epoch 5/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 143s 186ms/step - accuracy: 0.8112 - loss: 0.6308 - val_accuracy: 0.3332 - val_loss: 3.6751
Epoch 6/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 107s 179ms/step - accuracy: 0.8543 - loss: 0.4785 - val_accuracy: 0.3271 - val_loss: 4.1707
Epoch 7/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 144s 182ms/step - accuracy: 0.8835 - loss: 0.3870 - val_accuracy: 0.3355 - val_loss: 4.4052
Epoch 8/10
597/597 ━━━━━━━━━━━━━━━━━━━━ 144s 186ms/step - accuracy: 0.9138 -

In [ ]:
from sklearn.metrics import classification_report

# Évaluation sur les données de test
y_pred = (model.predict(X_val) > 0.5).astype("int32")
print(classification_report(y_val, y_pred))

150/150 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step
              precision    recall  f1-score   support

           0       0.40      0.44      0.42       300
           1       0.40      0.35      0.38       300
           2       0.57      0.54      0.56       300
           3       0.55      0.42      0.48       300
           4       0.48      0.43      0.45       310
           5       0.34      0.35      0.35       300
           6       0.31      0.20      0.24       300
           7       0.22      0.35      0.27       300
           8       0.24      0.21      0.22       200
           9       0.32      0.15      0.21        66
          10       0.28      0.11      0.16        61
          11       0.19      0.14      0.16        69
          12       0.26      0.22      0.24        65
          13       0.43      0.40      0.41       200
          14       0.39      0.34      0.36       200
          15       0.30      0.23      0.26        75
          16       0.61      0.32      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Test de prédiction sur une image personnalisée

from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_predict_image(image_path, img_size):

    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prédiction
    predictions = model.predict(img_array)

    # indice de classe prédite
    predicted_class = np.argmax(predictions, axis=1)

    # nom de la classe
    class_labels = list(generator.class_indices.keys())
    predicted_label = class_labels[predicted_class[0]]

    print(f"Image prédite comme étant de la classe : {predicted_label}")


image_path = "/content/dataset/ImageTest/Glazed-Donut.jpg"
load_and_predict_image(image_path, img_size)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Image prédite comme étant de la classe : Donut
